### Import dataset

In [1]:
import pandas as pd
import numpy as np

In [17]:
gas = pd.read_csv("C:/Users/aladesuru/Downloads/Natural_Gas_Consumption_by_ZIP_Code_-_2010_20240521.csv")
gas.head()

,Zip Code,Building type (service class,Consumption (therms),Utility/Data Source,Latitude,Longitude
0,10451,Commercial,"8,071,587",ConEd,40.820696,-73.923841
1,10468,Residential,"907,812",ConEd,40.869693,-73.898927
2,11212,Institutional,"1,441,719",National Grid,40.662797,-73.912985
3,11420,Small residential,"9,694,601",National Grid,40.673345,-73.817707
4,11694,Commercial,"2,058,660",National Grid,40.576578,-73.850777


In [18]:
# Check data types of gas accounts dataframe

gas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Zip Code                      964 non-null    int64  
 1   Building type (service class  964 non-null    object 
 2    Consumption (therms)         955 non-null    object 
 3   Utility/Data Source           964 non-null    object 
 4   Latitude                      964 non-null    float64
 5   Longitude                     964 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 45.3+ KB


In [19]:
missing_values = gas[gas.isnull().any(axis=1)]
gas.isnull().any(axis=1).sum()

9

In [20]:
gas = gas.dropna()
gas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 955 entries, 0 to 963
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Zip Code                      955 non-null    int64  
 1   Building type (service class  955 non-null    object 
 2    Consumption (therms)         955 non-null    object 
 3   Utility/Data Source           955 non-null    object 
 4   Latitude                      955 non-null    float64
 5   Longitude                     955 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 52.2+ KB


## Import folium package and map types.<br/><br/> We will import Choropleth, HeatMap, Marker, MarkerCluster, and Circle Maps

In [13]:
import folium # html mapping package
from folium import Choropleth, Circle, Marker 
from folium.plugins import HeatMap, MarkerCluster

## Marker Map

### Display blank basemap using folium

In [27]:
map1 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', zoom_start = 8)
map1

### Add markers to basemap using Marker in Folium using folium.Marker() and a loop for each marker. <br/> <br/>Each marker represents a different gas account

In [28]:
# iterate through each row
for idx, row in gas.iterrows():
    
# Add marker for each gas accounts 
    Marker([row['Latitude'], row['Longitude']]). add_to(map1) 
map1

### There are too many markers to get a good idea of where our accounts are concentrated.<br/><br/> With a MarkerCluster Map we can get a better idea of where our accounts are located since accounts in close proximity to one another are grouped together.

## MarkerCluster Map

In [29]:
# Define new map

map2 = folium.Map([np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', zoom_start = 8)

#Define MarkerCluster object

mc = MarkerCluster(name = 'Clustered Accounts')

# Loop through gas accounts DataFrame and add gas accounts to MarkerCluster objects
for idx, row in gas.iterrows():
    mc.add_child(Marker([row['Latitude'], row['Longitude']]))
    
map2.add_child(mc)

map2


### If we go back to the dataset we can see that the gas accounts are either coming from National Grid or ConEd.<br/><br/> How can we visualize our National Grid accounts seperate from our ConEd accounts?<br/><br/>One way to do this is by creating a HeatMap

In [8]:
gas.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,Latitude,Longitude
0,11109,Commercial,45899.0,ConEd,40.744415,-73.957702
1,11429,Commercial,755.0,ConEd,40.709913,-73.738640
2,11226,Industrial,65835.0,National Grid,40.646505,-73.957190
3,10314,Institutional,2294516.0,National Grid,40.596490,-74.165991
4,11223,Commercial,2376036.0,National Grid,40.596940,-73.973311


In [9]:
gas['Utility/Data Source'].value_counts()

ConEd            518
National Grid    446
Name: Utility/Data Source, dtype: int64

## HeatMap

In [33]:
# Define new map

map3 = folium.Map([np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', zoom_start = 8)

# Define HeatMap object for National Grid accounts

HeatMap(gas[gas['Utility/Data Source'] == 'National Grid'][['Latitude', 'Longitude']], radius = 10,
        gradient = {.5: 'purple', .65: 'yellow', 1: 'white'}, name = 'Heat Map of National Grid Accounts').add_to(map3)

# Define HeatMap object for ConEd accounts 

HeatMap(gas[gas['Utility/Data Source'] == 'ConEd'][['Latitude', 'Longitude']], radius = 10,
        gradient = {.5: 'black', .65: 'red', 1: 'gold'}, name = 'ConEd').add_to(map3)

# Add layer control to toggle between National Grid and ConEd accounts

folium.LayerControl().add_to(map3)

# Display Map

map3

## Bubble Map

### If we go back again to the to the dataset we can see that the account types are in the Building (service class) column. 

In [11]:
gas.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,Latitude,Longitude
0,11109,Commercial,45899.0,ConEd,40.744415,-73.957702
1,11429,Commercial,755.0,ConEd,40.709913,-73.738640
2,11226,Industrial,65835.0,National Grid,40.646505,-73.957190
3,10314,Institutional,2294516.0,National Grid,40.596490,-74.165991
4,11223,Commercial,2376036.0,National Grid,40.596940,-73.973311


In [12]:
# value counts for Building type

gas['Building type (service class)'].value_counts()

Commercial           338
Residential          198
Large residential    165
Small residential     96
Institutional         86
Industrial            81
Name: Building type (service class), dtype: int64

### How could we visualize each of these account types using a Bubble Map?

In [34]:
# Define a new map

map4 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap', 
                  zoom_start = 10)

# Color function 

def bubble_color(val):
    if val == 'National Grid':
        return 'purple'
    else:
        return 'red'


# Add Bubble layer to map using folium Circle function

for i in range(len(gas)):
    Circle(location = [gas.iloc[i]['Latitude'], gas.iloc[i]['Longitude']], radius = 20, 
           color = bubble_color(gas.iloc[i]['Utility/Data Source'])).add_to(map4)


# Display map

map4

## Choropleth Maps

### Choropleth maps allow for pre-defined areas to be colored or patterned in proportion to a statisitical variable. For example, you could check for average housing values by state in a map where darker areas would have higher prices.<br/> <br/>Let's say we wanted to know which borough uses the most and which one uses the least.<br/><br/>We could figure this out with a spatial join between the borough shapefiles and 

### import geodata packages

In [35]:
import geopandas as gpd # Pandas GeodDataFrame
from geopandas.tools import sjoin # Join shapefiles to coordinate data
from shapely.geometry import Point # Geopoints

### Create a GeoSeries to join latitudes and longitudes as points

In [36]:
# Define GeoSeries

points = gpd.GeoSeries(gas.apply(lambda x: Point(x['Longitude'], x['Latitude']), axis = 1), crs = {'init': 'epsg:4326'})

# Print GeoSeries

points.head()

c:\Users\aladesuru\.virtualenvs\Lab-wJx68UC_\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


0    POINT (-73.92384 40.82070)
1    POINT (-73.89893 40.86969)
2    POINT (-73.91298 40.66280)
3    POINT (-73.81771 40.67335)
4    POINT (-73.85078 40.57658)
dtype: geometry

In [37]:
# Make gas accounts GeoDataFrame
 
gas_gdf = gpd.GeoDataFrame(gas.drop(['Latitude', 'Longitude'], axis = 1) , geometry = points)
gas_gdf.head()

,Zip Code,Building type (service class,Consumption (therms),Utility/Data Source,geometry
0,10451,Commercial,"8,071,587",ConEd,POINT (-73.92384 40.82070)
1,10468,Residential,"907,812",ConEd,POINT (-73.89893 40.86969)
2,11212,Institutional,"1,441,719",National Grid,POINT (-73.91298 40.66280)
3,11420,Small residential,"9,694,601",National Grid,POINT (-73.81771 40.67335)
4,11694,Commercial,"2,058,660",National Grid,POINT (-73.85078 40.57658)


### We want to add the Neighborhood Tabulation Areas to our Map, but we need to extract them from a zipfile using the zipfile package

### Get shapefiles from zip folder

In [31]:
# import zipfile package

import zipfile

# open zipfile with zipfile package

with zipfile.ZipFile('Your file path/Borough Boundaries (Water Areas Included).zip', 'r') as my_zip:
    print(my_zip.namelist())
    my_zip.extractall('files')

['geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.dbf', 'geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.shp', 'geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.shx', 'geo_export_49512982-8e61-4a09-91ce-6a6640b8d43c.prj']


### Store filenames from zip folder as variables

In [32]:
# Store filenames for zipfile files

dbf, shp, shx, prj = my_zip.namelist()

# Your local path

local_path = 'files/'

In [33]:
# read in borough shapefiles using read_file module

boroughs = gpd.read_file(local_path + shp)
boroughs.head()

,boro_code,boro_name,shape_area,shape_leng,geometry
0,1.0,Manhattan,9.442947e+08,203803.216852,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,2.0,Bronx,1.598380e+09,188054.198841,"POLYGON ((-73.86477 40.90201, -73.86305 40.901..."
2,3.0,Brooklyn,2.684411e+09,234928.658563,"POLYGON ((-73.92722 40.72533, -73.92654 40.724..."
3,4.0,Queens,3.858050e+09,429586.630985,"POLYGON ((-73.77896 40.81171, -73.76371 40.793..."
4,5.0,Staten Island,2.539686e+09,212213.139971,"POLYGON ((-74.05581 40.64971, -74.05619 40.639..."


### Make sure boroughs shapefile and GeoDataFrame have same CRS

In [36]:
# Check crs for gas accounts

gas_gdf.crs

{'init': 'epsg: 4326'}

In [37]:
# Check crs  for boroughs shapefile

boroughs.crs

{'init': 'epsg:4326'}

In [38]:
# Change crs so both match 

boroughs.crs = {'init': 'epsg: 4326'}

### Join our gas accounts to the boroughs shapefile using an inner spatial join

In [39]:
# sum join gas accounts to boroughs shapefile

gas_sum_join = gpd.sjoin().groupby().sum()

# mean join gas accounts to boroughs shapefile

gas_mean_join = gpd.sjoin().groupby().mean()

# count join gas accounts to boroughs shapefile

gas_counts_join = gpd.sjoin().groupby().count()

In [6]:
gas_sum_join['Consumption (therms)']

NameError: name 'gas_sum_join' is not defined

In [41]:
gas_mean_join['Consumption (therms)']

boro_name
Bronx            2.663300e+06
Brooklyn         5.005716e+06
Manhattan        2.244163e+06
Queens           2.172726e+06
Staten Island    3.167656e+06
Name: Consumption (therms), dtype: float64

In [42]:
boroughs

,boro_code,boro_name,shape_area,shape_leng,geometry
0,1.0,Manhattan,9.442947e+08,203803.216852,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ..."
1,2.0,Bronx,1.598380e+09,188054.198841,"POLYGON ((-73.86477 40.90201, -73.86305 40.901..."
2,3.0,Brooklyn,2.684411e+09,234928.658563,"POLYGON ((-73.92722 40.72533, -73.92654 40.724..."
3,4.0,Queens,3.858050e+09,429586.630985,"POLYGON ((-73.77896 40.81171, -73.76371 40.793..."
4,5.0,Staten Island,2.539686e+09,212213.139971,"POLYGON ((-74.05581 40.64971, -74.05619 40.639..."


In [43]:
# Define new map 

map5 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap',
                  zoom_start = 10)

# Add Choropleth Layer

Choropleth(geo_data = boroughs.__geo_interface__, data = gas_mean_join['Consumption (therms)'], 
          key_on = 'feature.properties.boro_name', fill_color = 'BuPu', legend_name = 'Average Therms Per Borough', 
          name = 'Borough Shapefiles').add_to(map5)

# Add Layer control

folium.LayerControl().add_to(map5)

# Display map

map5

## Combination Map

In [44]:
# Define new map 

map6 = folium.Map(location = [np.mean(gas['Latitude']), np.mean(gas['Longitude'])], tiles = 'OpenStreetMap',
                  zoom_start = 10)

# Add Choropleth Layer

Choropleth()

# Add MarkerCluster Layer

mc2 = MarkerCluster()

for idx, row in gas.iterrows():


# Add Layer control



# Display map
